Figure 1: Flowchart

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import re

# Set larger figure size for better readability
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 10

# Load the main dataset
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Masterarbeit Analgesie_24.07.2025.xlsx'
data_df = pd.read_excel(data_path)

# Load the medication data
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Liste Notärzte-1.xlsx'
medic_df = pd.read_excel(medic_data_path)

# Merge physician data
medic_df = medic_df.drop_duplicates('Mitglieder mit Einsatzfunktion')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

print(f"Total dataset size: {len(data_df)} patients")
print(f"Medication dataset size: {len(medic_df)} physicians")

# Display basic info about the data
print(f"\nAge distribution:")
print(f"Adult (≥16): {(data_df['Alter '] >= 16).sum()}")
print(f"Pediatric (<16): {(data_df['Alter '] < 16).sum()}")
print(f"Missing age: {data_df['Alter '].isna().sum()}")

In [ ]:
# Process medication data (using same approach as figure2)
def process_medications(df):
    """Process medication data to identify analgesic use"""
    import re
    
    df_copy = df.copy()
    
    # Initialize medication dose columns
    df_copy['morphine_dose'] = 0.0
    df_copy['fentanyl_dose'] = 0.0
    df_copy['ketamine_dose'] = 0.0
    df_copy['esketamine_dose'] = 0.0
    
    for i, row in df_copy.iterrows():
        medications = row.get('Alle Medikamente', '')
        if pd.notna(medications) and medications:
            medications = str(medications)
            
            # Fentanyl
            fentanyl_matches = re.findall(r'Fentanyl[^;]*?(\d+(?:\.\d+)?)mcg', medications, re.IGNORECASE)
            fentanyl_total = sum(float(dose) for dose in fentanyl_matches)
            df_copy.at[i, 'fentanyl_dose'] = fentanyl_total
            
            # Ketamine (not Esketamine)
            ketamine_matches = re.findall(r'(?<!Es)ketamin[^;]*?(\d+(?:\.\d+)?)mg', medications, re.IGNORECASE)
            ketamine_total = sum(float(dose) for dose in ketamine_matches)
            df_copy.at[i, 'ketamine_dose'] = ketamine_total
            
            # Esketamine
            esketamine_matches = re.findall(r'Esketamin[^;]*?(\d+(?:\.\d+)?)mg', medications, re.IGNORECASE)
            esketamine_total = sum(float(dose) for dose in esketamine_matches)
            df_copy.at[i, 'esketamine_dose'] = esketamine_total
            
            # Morphine
            morphine_matches = re.findall(r'Morphin[^;]*?(\d+(?:\.\d+)?)mg', medications, re.IGNORECASE)
            morphine_total = sum(float(dose) for dose in morphine_matches)
            df_copy.at[i, 'morphine_dose'] = morphine_total
    
    # Create binary indicators
    df_copy['any_opiate_given'] = (df_copy['morphine_dose'] > 0) | (df_copy['fentanyl_dose'] > 0)
    df_copy['any_ketamine_given'] = (df_copy['ketamine_dose'] > 0) | (df_copy['esketamine_dose'] > 0)
    df_copy['any_analgesic'] = df_copy['any_opiate_given'] | df_copy['any_ketamine_given']
    
    return df_copy

# Process medications
print("Processing medications...")
data_df = process_medications(data_df)

# Calculate flowchart numbers for both populations
def calculate_flowchart_numbers(df, population_name, age_filter=None):
    """Calculate numbers for flowchart based on inclusion/exclusion criteria"""
    
    if age_filter is not None:
        df_pop = df[age_filter].copy()
    else:
        df_pop = df.copy()
    
    total_cases = len(df_pop)
    
    # Apply exclusions similar to original study
    # Age exclusions already applied by age_filter
    
    # Check for GCS data
    gcs_col = None
    for col in ['GCS', 'GCS vor Ort', 'GCS_vor_Ort']:
        if col in df_pop.columns:
            gcs_col = col
            break
    
    # Calculate exclusions
    if gcs_col:
        low_gcs = (df_pop[gcs_col] < 13).sum() if df_pop[gcs_col].notna().any() else 0
    else:
        low_gcs = 0
    
    # Missing NRS/VAS data
    missing_vas_scene = df_pop['VAS_on_scene'].isna().sum()
    missing_vas_arrival = df_pop['VAS_on_arrival'].isna().sum()
    missing_vas = missing_vas_scene + missing_vas_arrival
    
    # After exclusions - patients with complete data
    complete_data = df_pop.dropna(subset=['VAS_on_scene', 'VAS_on_arrival'])
    
    # NRS ≤ 3 at scene
    nrs_low_scene = (complete_data['VAS_on_scene'] <= 3).sum()
    
    # NRS > 3 at scene 
    nrs_high_scene = (complete_data['VAS_on_scene'] > 3).sum()
    
    # Among NRS > 3 at scene, split by analgesic use
    high_pain_patients = complete_data[complete_data['VAS_on_scene'] > 3]
    
    analgesic_given = high_pain_patients['any_analgesic'].sum()
    no_analgesic = len(high_pain_patients) - analgesic_given
    
    # Among analgesic group - successful pain management (NRS ≤ 3 at arrival)
    analgesic_patients = high_pain_patients[high_pain_patients['any_analgesic']]
    successful_pain_mgmt = (analgesic_patients['VAS_on_arrival'] <= 3).sum()
    unsuccessful_pain_mgmt = len(analgesic_patients) - successful_pain_mgmt
    
    # Among no analgesic group - pain outcomes
    no_analgesic_patients = high_pain_patients[~high_pain_patients['any_analgesic']]
    no_analgesic_success = (no_analgesic_patients['VAS_on_arrival'] <= 3).sum()
    no_analgesic_persistent = len(no_analgesic_patients) - no_analgesic_success
    
    # Final outcomes
    total_nrs_low_final = successful_pain_mgmt + no_analgesic_success + nrs_low_scene
    total_nrs_high_final = unsuccessful_pain_mgmt + no_analgesic_persistent
    
    results = {
        'population': population_name,
        'total_cases': total_cases,
        'exclusions': {
            'low_gcs': low_gcs,
            'missing_vas': missing_vas
        },
        'after_exclusions': len(complete_data),
        'nrs_low_scene': nrs_low_scene,
        'nrs_high_scene': nrs_high_scene,
        'analgesic_given': analgesic_given,
        'no_analgesic': no_analgesic,
        'analgesic_success': successful_pain_mgmt,
        'analgesic_failure': unsuccessful_pain_mgmt,
        'no_analgesic_success': no_analgesic_success,
        'no_analgesic_persistent': no_analgesic_persistent,
        'final_nrs_low': total_nrs_low_final,
        'final_nrs_high': total_nrs_high_final
    }
    
    return results

# Calculate for both populations
adult_numbers = calculate_flowchart_numbers(data_df, "Adults", data_df['Alter '] >= 16)
pediatric_numbers = calculate_flowchart_numbers(data_df, "Pediatric", data_df['Alter '] < 16)

print("\\nAdult Population Flowchart Numbers:")
for key, value in adult_numbers.items():
    print(f"  {key}: {value}")

print("\\nPediatric Population Flowchart Numbers:")
for key, value in pediatric_numbers.items():
    print(f"  {key}: {value}")

In [ ]:
def create_flowchart(numbers, title):
    """Create a flowchart based on the calculated numbers"""
    
    fig, ax = plt.subplots(1, 1, figsize=(14, 16))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 20)
    ax.axis('off')
    
    # Define colors
    colors = {
        'total': '#E8E8E8',
        'exclusion': '#D3D3D3', 
        'scene_low': '#90EE90',
        'scene_high': '#FFE4B5',
        'analgesic': '#87CEEB',
        'no_analgesic': '#F0E68C',
        'success': '#98FB98',
        'failure': '#FFA07A',
        'final_success': '#90EE90',
        'final_failure': '#FFA07A'
    }
    
    # Helper function to create boxes
    def create_box(x, y, w, h, text, color, textcolor='black', fontsize=10):
        box = FancyBboxPatch((x, y), w, h,
                           boxstyle="round,pad=0.05",
                           facecolor=color,
                           edgecolor='black',
                           linewidth=1.5)
        ax.add_patch(box)
        # Replace \\n with actual newlines for proper rendering
        formatted_text = text.replace('\\n', '\n')
        ax.text(x + w/2, y + h/2, formatted_text, ha='center', va='center', 
                fontsize=fontsize, fontweight='bold', color=textcolor, wrap=True)
    
    # Helper function to draw arrows
    def draw_arrow(x1, y1, x2, y2, color='black'):
        ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                   arrowprops=dict(arrowstyle='->', color=color, lw=2))
    
    # Title
    ax.text(5, 19.5, f'Figure 1: {title} Population Flowchart', 
            ha='center', va='center', fontsize=16, fontweight='bold')
    
    # 1. Total cases
    create_box(3.5, 17.5, 3, 1, 
               f'Total cases\nn={numbers["total_cases"]:,}',
               colors['total'], fontsize=12)
    
    # Arrow down
    draw_arrow(5, 17.4, 5, 16.6)
    
    # 2. Exclusions
    exclusion_text = f'Exclusions:\nMissing NRS: {numbers["exclusions"]["missing_vas"]:,}'
    if numbers["exclusions"]["low_gcs"] > 0:
        exclusion_text += f'\nLow GCS: {numbers["exclusions"]["low_gcs"]:,}'
    
    create_box(3.5, 15.5, 3, 1.5,
               exclusion_text,
               colors['exclusion'], fontsize=10)
    
    # Arrow down
    draw_arrow(5, 15.4, 5, 14.6)
    
    # 3. After exclusions with NRS ≤3 at scene branch
    create_box(3.5, 13.5, 3, 1,
               f'After exclusions\nn={numbers["after_exclusions"]:,}',
               colors['total'], fontsize=11)
    
    # NRS ≤3 at scene (side branch)
    create_box(7.5, 13.5, 2.2, 1,
               f'NRS ≤3 at scene\nn={numbers["nrs_low_scene"]:,}\n({numbers["nrs_low_scene"]/numbers["after_exclusions"]*100:.0f}%)',
               colors['scene_low'], fontsize=9)
    
    # Arrows
    draw_arrow(5, 13.4, 5, 12.6)  # Main flow down
    draw_arrow(6.6, 14, 7.4, 14)  # Side branch right
    
    # 4. NRS > 3 at scene
    create_box(3.5, 11.5, 3, 1,
               f'NRS > 3 at scene\nn={numbers["nrs_high_scene"]:,}\n({numbers["nrs_high_scene"]/numbers["after_exclusions"]*100:.0f}%)',
               colors['scene_high'], fontsize=11)
    
    # Arrow down
    draw_arrow(5, 11.4, 5, 10.8)
    
    # 5. Split into analgesic vs no analgesic
    # Analgesic given
    create_box(1, 9.5, 2.5, 1.2,
               f'Analgesic\nn={numbers["analgesic_given"]:,}\n({numbers["analgesic_given"]/numbers["nrs_high_scene"]*100:.0f}%)',
               colors['analgesic'], fontsize=10)
    
    # No analgesic
    create_box(6.5, 9.5, 2.5, 1.2,
               f'No analgesic\nn={numbers["no_analgesic"]:,}\n({numbers["no_analgesic"]/numbers["nrs_high_scene"]*100:.0f}%)',
               colors['no_analgesic'], fontsize=10)
    
    # Arrows from NRS > 3 to both branches
    draw_arrow(4.2, 10.5, 2.5, 10.8)  # To analgesic
    draw_arrow(5.8, 10.5, 6.5, 10.8)  # To no analgesic
    
    # 6. Outcomes for analgesic group
    # Success
    create_box(0.5, 7.5, 2, 1.2,
               f'NRS ≤3 at arrival\nn={numbers["analgesic_success"]:,}\n({numbers["analgesic_success"]/numbers["analgesic_given"]*100:.0f}%)',
               colors['success'], fontsize=9)
    
    # Failure
    create_box(3, 7.5, 2, 1.2,
               f'NRS > 3 at arrival\nn={numbers["analgesic_failure"]:,}\n({numbers["analgesic_failure"]/numbers["analgesic_given"]*100:.0f}%)',
               colors['failure'], fontsize=9)
    
    # 7. Outcomes for no analgesic group
    # Success
    create_box(5.5, 7.5, 2, 1.2,
               f'NRS ≤3 at arrival\nn={numbers["no_analgesic_success"]:,}\n({numbers["no_analgesic_success"]/numbers["no_analgesic"]*100:.0f}%)',
               colors['success'], fontsize=9)
    
    # Persistent pain
    create_box(8, 7.5, 2, 1.2,
               f'NRS > 3 at arrival\nn={numbers["no_analgesic_persistent"]:,}\n({numbers["no_analgesic_persistent"]/numbers["no_analgesic"]*100:.0f}%)',
               colors['failure'], fontsize=9)
    
    # Arrows from treatment groups to outcomes
    draw_arrow(1.8, 9.4, 1.5, 8.8)   # Analgesic to success
    draw_arrow(2.8, 9.4, 4, 8.8)     # Analgesic to failure
    draw_arrow(7.2, 9.4, 6.5, 8.8)   # No analgesic to success  
    draw_arrow(8.2, 9.4, 9, 8.8)     # No analgesic to persistent
    
    # 8. Final outcomes
    # Final NRS ≤3 (combining all success paths)
    create_box(2, 5.5, 2.5, 1.2,
               f'Final NRS ≤3\nn={numbers["final_nrs_low"]:,}\n({numbers["final_nrs_low"]/numbers["after_exclusions"]*100:.0f}%)',
               colors['final_success'], fontsize=10)
    
    # Final NRS > 3 
    create_box(5.5, 5.5, 2.5, 1.2,
               f'Final NRS > 3\nn={numbers["final_nrs_high"]:,}\n({numbers["final_nrs_high"]/numbers["after_exclusions"]*100:.0f}%)',
               colors['final_failure'], fontsize=10)
    
    # Arrows to final outcomes
    draw_arrow(1.5, 7.4, 2.8, 6.8)   # Success paths to final success
    draw_arrow(6.5, 7.4, 4, 6.8)     # Success paths to final success
    draw_arrow(8.5, 13.4, 3.2, 6.8)  # Initial low NRS to final success
    
    draw_arrow(4, 7.4, 6.2, 6.8)     # Failure paths to final failure
    draw_arrow(9, 7.4, 7.2, 6.8)     # Persistent pain to final failure
    
    plt.tight_layout()
    plt.show()

# Create flowcharts for both populations
print("Creating Adult Population Flowchart...")
create_flowchart(adult_numbers, "Adult")

print("\nCreating Pediatric Population Flowchart...")
create_flowchart(pediatric_numbers, "Pediatric")